In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
 
import warnings
warnings.filterwarnings('ignore') 
# 不发出警告

from bokeh.io import output_notebook
output_notebook()
# 导入notebook绘图模块
 
from bokeh.plotting import figure,show
from bokeh.models import ColumnDataSource
# 导入图表绘制、图标展示模块
# 导入ColumnDataSource模块

import os
os.chdir('E:/学习资料/python数据分析/项目资料/项目09中国姓氏排行研究/')


from bokeh.models import HoverTool
from bokeh.core.properties import value

Loading BokehJS ...

In [5]:
'''
(1)数据加载及清洗
'''

#数据读取
df1 = pd.read_csv('data01.csv',encoding = 'utf8')
df2 = pd.read_csv('data02.csv',encoding = 'utf8')
df_city = pd.read_excel('中国行政代码对照表.xlsx')

#数据清洗
df = pd.concat([df1,df2])
df = pd.merge(df,df_city,left_on = '户籍地城市编号',right_on = '行政编码')

del df['户籍地城市编号']
del df['行政编码']
df['工作地'] = df['工作地'].str[:15]

In [19]:
'''
(2)提取工作地的省市区县
'''

df['工作地_省'] = df['工作地'].str.split('省').str[0]

df['工作地_市'] = df['工作地'].str.split('省').str[1].str.split('市').str[0]
df['工作地_市'][df['工作地_省'].str.len()>5] = df['工作地_省'].str.split('市').str[0]

df['工作地_区县'] = ''
df['工作地_区县'][(df['工作地_市'].str.len()<5)&(df['工作地'].str.contains('区'))] = df['工作地'].str.split('市').str[1].str.split('区').str[0]+'区'
df['工作地_区县'][(df['工作地_市'].str.len()>5)&(df['工作地'].str.contains('区'))] = df['工作地'].str.split('区').str[0]+'区'

df['工作地_区县'][(df['工作地_市'].str.len()<5)&(df['工作地'].str.contains('县'))] = df['工作地'].str.split('市').str[1].str.split('县').str[0]+'县'
df['工作地_区县'][(df['工作地_市'].str.len()>5)&(df['工作地'].str.contains('县'))] = df['工作地'].str.split('县').str[0]+'县'

df['工作地_省'][df['工作地_省'].str.len()>5] = '未识别'
df['工作地_市'][df['工作地_市'].str.len()>5] = '未识别'
df['工作地_区县'][(df['工作地_区县'].str.len()>5)|(df['工作地_区县'].str.len()<2)] = '未识别'

df

,姓,工作地,省,市,区/县,lng,lat,工作地_省,工作地_市,工作地_区县
0,王,沈阳市皇姑区塔湾街83-15号,辽宁省,沈阳,皇姑区,123.415376,41.848913,未识别,沈阳,皇姑区
1,刘,虹口区四川北路1851号12,辽宁省,沈阳,皇姑区,123.415376,41.848913,未识别,未识别,虹口区
2,李,沈阳市皇姑区香炉山路3-1号2,辽宁省,沈阳,皇姑区,123.415376,41.848913,未识别,沈阳,皇姑区
3,王,沈阳市皇姑区崇山中路,辽宁省,沈阳,皇姑区,123.415376,41.848913,未识别,沈阳,皇姑区
4,刘,沈阳市皇姑区梅江东街11号4-,辽宁省,沈阳,皇姑区,123.415376,41.848913,未识别,沈阳,皇姑区
5,王,章武县章武镇,辽宁省,沈阳,皇姑区,123.415376,41.848913,未识别,未识别,章武县
6,张,沈阳市黄姑区金沙江街39号5-,辽宁省,沈阳,皇姑区,123.415376,41.848913,未识别,沈阳,黄姑区
7,刘,铁西区腾飞一街31号,辽宁省,沈阳,皇姑区,123.415376,41.848913,未识别,未识别,铁西区
8,阎,沈阳市皇姑区明廉路一72-4-,辽宁省,沈阳,皇姑区,123.415376,41.848913,未识别,沈阳,皇姑区
9,雷,沈阳市皇姑区,辽宁省,沈阳,皇姑区,123.415376,41.848913,未识别,沈阳,皇姑区


In [24]:
'''
(3)数据统计，找到姓氏TOP20
'''

name_count = df['姓'].value_counts()[:20]
result1_01 = pd.DataFrame({'count':name_count,'count_pre':name_count/name_count.sum()})

'''
（4）bokeh做联动柱状图
'''

from bokeh.models import HoverTool
from bokeh.layouts import gridplot

source = ColumnDataSource(data = result1_01)

#柱状图1
hover1 = HoverTool(tooltips = [('姓氏计数','@count')])
p1 = figure(plot_width = 800,plot_height = 250,
            title = '姓氏top20_计数',
            tools = [hover1,'wheel_zoom'],
            x_range = result1_01.index.tolist())
p1.vbar(x = 'index',top = 'count',source = source,
        width = 0.9,alpha = 0.7,color = 'red')

#柱状图2
hover2 = HoverTool(tooltips = [('姓氏占比','@count_pre')])
p2 = figure(plot_width = 800,plot_height = 250,
            title = '姓氏top20_占比',
            tools = [hover2,'wheel_zoom'],
            x_range = p1.x_range)
p2.vbar(x = 'index',top = 'count_pre',source = source,
        width = 0.9,alpha = 0.7,color = 'green')

p = gridplot([p1],[p2])
show(p)